In [1]:
# Special libraries used
!pip install pyreclab                     # For ALS based candidate generation
!pip install tf-nightly-gpu-2.0-preview   # For GPU usage, neural networks

In [0]:
import pyreclab as rec
import tensorflow as tf